# Homework 3: Neural Networks


This assignment is due on Moodle by **11:59pm on Monday March 15**. 
Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.
Your solutions to computational questions should include any specified Python code and results 
as well as written commentary on your conclusions.
Remember that you are encouraged to discuss the problems with your instructors and classmates, 
but **you must write all code and solutions on your own**. For a refresher on the course **Collaboration Policy** click [here](https://github.com/akkikiki/CSCI-4622-Machine-Learning-sp21/blob/main/info/syllabus.md#collaboration-policy).


**NOTES**: 

- Do **NOT** load or use any Python packages that are not available in Anaconda (Version: 2019.07) with Python 3.7. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Moodle.  Do not compress it using tar, rar, zip, etc. 
- In this homework, we will use [PyTorch](https://pytorch.org/) and PyTorch Lightning to implement a classifier. First upgrade your `pip` package manager to the latest version (Version >19.0). Then install the current stable release for [PyTorch](https://pytorch.org/get-started/locally/) (version: 1.6.0) as the backend for PyTorch Lightning. Then install [PyTorch Lighntning](https://pytorch-lightning.readthedocs.io/en/0.9.0/) (make sure to isntall version: 0.9.0 for this HW).

```
pip install --upgrade pip

pip install torch

pip install pytorch-lightning==0.9.0

```

It is **highly recommended** you install the CPU-only version of PyTorch if your'e unfamiliar with the process of installing CUDA libraries for these packages. If you choose to install a GPU version, ensure you're code runs without GPU support.

**Acknowledgment** : Chris Ketelsen, Chenhao Tan

**Please put your name and cuidentity username.**

**Name**: Rees LaBree

**Identity Key**: rela3138

In [1]:
import math
import pickle
import gzip
import numpy as np
import pandas
import matplotlib.pylab as plt
%matplotlib inline

# Test PyTorch installation
import torch 
import pytorch_lightning as pl

pl.seed_everything(1234)

1234

[20 points] Problem 1 - Single-Layer and Multilayer Perceptron Learning
---

### Part 1 [10 points] 
Consider learning the following concepts with either a single-layer or multilayer perceptron where all hidden and output neurons utilize the *indicator* activation functions. For each of the following concepts, state whether the concept can be learned by a single-layer perceptron. Briefly justify your response by providing weights, biases, and the *indicator* activation functions if applicable:

- $\; \texttt{NOT } x_1$

- $\; x_1 \texttt{ NAND } x_2$

- $\; x_1 \texttt{ XNOR } x_2$ (output 1 when $x_1 = x_2$ and 0 otherwise)

NOT and NAND linearly seperable which is why it can be learned by a single layer perceptron, XNOR is not and thus cannot.
- NOT - weights = <-1>, bias = <0>, If wx+b > 0, y' = 1
- NAND - weights = <-1, -1>, bias = <2>, If wx1+wx2+b > 0, y' = 1
- XNOR could not be done with a single layer perceptron, it would require combinging multiple layers.

### Part 2 [10 points] 

Determine an architecture and specific values of the weights and biases in a single-layer or multilayer perceptron with *indicator* activation functions that can learn $x_1 \texttt{ XNOR } x_2$. Make a truth table of $x_1$, $x_2$, and $x_1 \texttt{ XNOR } x_2$, describe your perceptron's architecture, and state your weight matrices and bias vectors in Markdown below. Then demonstrate that your solution is correct by implementing forward propagation for your network in Python and showing that it produces the correct boolean output values for each of the four possible combinations of $x_1$ and $x_2$.

XNOR is the same as an AND gate or'd with a NOR gate. Both AND and NOR can be accomplished with single layer perceptrons. As such we will feed b, w1x1 and w2x2 to AND and NOR, and then feed the product of the new weights and outputs and a new bias to OR to generate Y.
- WEIGHTS: $w_{NOR} = <-1, -1>$, $w_{AND} = <1, 1>$, $w_{OR} = <2, 2>$
- BIAS: $b_{AND} = b_{OR} = -1, b_{NOR} = 1$
$$
\begin{array}{|c|c|c|}
\hline
x1& x2 & out\\ \hline
0 & 0 & 1\\ \hline
1 & 0 & 0\\ \hline
0 & 1 & 0\\ \hline
1 & 1 & 1\\ \hline
\end{array}
$$

In [2]:
# YOUR CODE HERE
def XNOR(x1, x2):
    w_AND = [1, 1]
    w_NOR = [-1, -1]
    w_OR = [2, 2]
    bias_AND_OR = -1
    bias_NOR = 1

    AND = w_AND[0]*x1 + w_AND[1]*x2 + bias_AND_OR
    if AND <= 0:
        AND = 0
    else:
        AND = 1
    
    NOR = w_NOR[0]*x1 + w_NOR[1]*x2 + bias_NOR
    if NOR <= 0:
        NOR = 0
    else:
        NOR = 1
    
    result = w_OR[0]*AND + w_OR[1]*NOR + bias_AND_OR
    if result <= 0:
        result = 0
    else:
        result = 1
    
    
    return result


a = XNOR(0, 0)
b = XNOR(0, 1)
c = XNOR(1, 0)
d = XNOR(1, 1)

print(" x1 | x2 | out")
print("----+----+-----")
print("  0 |  0 |  %d" % (a))
print("----+----+-----")
print("  0 |  1 |  %d" % (b))
print("----+----+-----")
print("  1 |  0 |  %d" % (c))
print("----+----+-----")
print("  1 |  1 |  %d" % (d))

 x1 | x2 | out
----+----+-----
  0 |  0 |  1
----+----+-----
  0 |  1 |  0
----+----+-----
  1 |  0 |  0
----+----+-----
  1 |  1 |  1


[25 points] Problem 2 - Back propagation
---

In this problem you will gain some intuition about why training deep neural networks can be very time consuming.  Consider training a chain-like neural network: 

![chain-like nn](figs/chain_net.png)

Note that this network has three weights $W^1, W^2, W^3$ and three biases $b^1, b^2,$ and $b^3$ (for this problem you can think of each parameter as a single value or as a $1 \times 1$ matrix). Suppose that each hidden and output neuron is equipped with a sigmoid activation function and the loss function is given by 

$$
\ell(y, a^3) = \frac{1}{2}(y - a^3)^2  
$$

where $a^3$ is the value of the activation at the output neuron and $y \in \{0,1\}$ is the true label associated with the training example. 

### Part 1 [5 points]
Suppose each of the weights is initialized to $W^k = -1.0$ and each bias is initialized to $b^k = 0.5$.  Use forward propagation to find the activities and activations associated with each hidden and output neuron for the training example $(x, y) = (0.5,0)$. Show your work. 

- $a^0 = x = .5$
- $z^1 = -1.0a^0+0.5 = -1.0 (.5) + 0.5 = 0$
- $a^1 = \text{sigmoid}(z^1) = .5$
- $\ell(y, a^1) = \frac{1}{2}(y-a^1)^2 = \frac{1}{8}$
$$$$
$$$$
- $a^1 = .5$
- $z^2 = -1.0a^1+0.5 = -1.0 (.5) + 0.5 = 0$
- $a^2 = \text{sigmoid}(z^1) = .5$
- $\ell(y, a^2) = \frac{1}{2}(y-a^2)^2 = \frac{1}{8}$
$$$$
$$$$
- $a^2 = .5$
- $z^3 = -1.0a^2+0.5 = -1.0 (.5) + 0.5 = 0$
- $a^3 = \text{sigmoid}(z^3) = .5$
- $\ell(y, a^3) = \frac{1}{2}(y-a^3)^2 = \frac{1}{8}$

### Part 2 [5 points]:
Use Back-Propagation to compute the weight and bias derivatives $\partial \ell / \partial W^k$ and $\partial \ell / \partial b^k$ for $k=1, 2, 3$.  Show all work. 

$
\delta^3 = \frac{\partial \ell}{\partial a^L_j} \cdot g'(z^3) = (a^3 - y) \sigma '(z^3) = \frac{1}{2}\sigma '(z^3)\\
$
$$$$
for K = 3
$
\frac{\partial \ell}{\partial W^3} = \delta^3(a^2)^T = \frac{1}{2}\sigma '(z^3)\cdot \frac{1}{2} = \frac{1}{4}\sigma '(z^3) \Rightarrow \boxed{\frac{1}{16}}\\
\frac{\partial \ell}{\partial b^\mathcal{k}} = \delta^3 = \frac{1}{2}\sigma '(z^3)\Rightarrow\boxed{\frac{1}{8}}\\
$
$$$$
for K = 2
$
\delta^2 = (W^\ell)^T\delta^3\cdot g'(z^2) = -1\cdot\frac{1}{2}\sigma '(z^3)\cdot \sigma'(z^2)\\
\frac{\partial \ell}{\partial W^2} = \delta^2(a^1)^T = -\frac{1}{2}\sigma '(z^3)\cdot \sigma'(z^2)\cdot(\frac{1}{2}) = -\frac{1}{4}\sigma '(z^3)\cdot \sigma'(z^2)\Rightarrow\boxed{-\frac{1}{64}}\\
\frac{\partial \ell}{\partial b^\mathcal{k}} = \delta^2 = -\frac{1}{2}\sigma '(z^3)\cdot \sigma'(z^2) \Rightarrow \boxed{-\frac{1}{32}}\\
$
$$$$
for K = 1
$
\delta^1 = (W^\ell)^T\delta^2\cdot g'(z^1) = \frac{1}{2}\sigma '(z^3)\cdot \sigma'(z^2)\cdot \sigma'(z^1)\\
\frac{\partial \ell}{\partial W^2} = \delta^2(a^1)^T = \frac{1}{2}\sigma '(z^3)\cdot \sigma'(z^2)\cdot \sigma'(z^1)\cdot(\frac{1}{2}) = \frac{1}{4}\sigma '(z^3)\cdot \sigma'(z^2)\cdot \sigma'(z^1)\Rightarrow\boxed{\frac{1}{256}}\\
\frac{\partial \ell}{\partial b^\mathcal{k}} = \delta^2 = \frac{1}{2}\sigma '(z^3)\cdot \sigma'(z^2)\cdot \sigma'(z^1)\Rightarrow\boxed{\frac{1}{128}}\\
$

### Part 3 [5 points]
Implement the following activation functions in NumPy:
* ReLU
* Sigmoid
* softmax
Please **do not use any other external libraries** for this such as sklearn.

In [3]:
def relu(x):
    # YOUR CODE HERE
    return max(0, x)

def sigmoid(x):
    # YOUR CODE HERE
    return (1/1+np.exp(-1*x))

def softmax(x):
    # YOUR CODE HERE
    sum_x = 0
    for x_i in x:
        sum_x += x_i
    return (np.exp(x)/sum_x)

In [4]:
# random tests for checking your impementation.
from tests import tests
tests.run_test_suite('prob 2.3', (relu, sigmoid, softmax))

TestRelu (tests.tests.TestActiv) ... ok
TestSigmoid (tests.tests.TestActiv) ... ok
TestSoftmax (tests.tests.TestActiv) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.019s

OK


### Part 4 [5 points] 
Implement the following Loss functions in NumPy:
* mean squared error
* mean absolute error
* hinge

Please **do not use any other external libraries** for this such as sklearn.

In [5]:
def mean_squared_error(yhat, y):
    # YOUR CODE HERE
    sum_i = 0.0
    for i, j in enumerate(yhat):
        sum_i += ((y[i]-j)**2)
    
    sum_i = sum_i / len(y)
    return sum_i
    
def mean_absolute_error(yhat, y):
    sum_i = 0.0
    for i, j in enumerate(yhat):
        sum_i += abs(y[i]-j)
    
    sum_i = sum_i / len(y)
    return sum_i

def hinge(yhat, y):
    # YOUR CODE HERE
    sum_i = 0
    for i, j in enumerate(yhat):
        sum_i += max(0.0, (1-y[i]*j))
        
    sum_i = sum_i / len(y)
    return sum_i

In [6]:
# random tests for checking your impementation.
from tests import tests
tests.run_test_suite('prob 2.4', (mean_squared_error, mean_absolute_error, hinge))

TestMSE (tests.tests.TestLoss) ... ok
TestMAE (tests.tests.TestLoss) ... ok
TestHinge (tests.tests.TestLoss) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.023s

OK


### Part 5 [5 points]
Explain the vanishing gradient problem. When would you observe this? Give an example.

By compressing the output of a gradient based approach to neural networks using certain activation functions (i.e. reducing to the range [0,1]), large changes in the data will be compressed to small, indiscernable changes and hence the gradient "vanishes". You would observe this when the neural networks are initialized with excessively large negative or positive weights. 

[23 Points] Problem 3 - Build a feedforward neural network in NumPy
---

In this problem you will implement a class representing a general feed-forward neural network that utilizes the sigmoid activation functions. Your tasks will be to implement forward propagation, prediction, back propagation, and a general train routine to learn the weights in your network via stochastic gradient descent.

The skeleton for the network class is below. Note that this class is almost identical to the one you worked with in the "hands-on neural network" in-class notebook, so you should look at there to remind yourself of the details. Scroll down to find more information about your tasks as well as unit tests. As with the previous problem, nothing in this problem should use any external libraries apart from numpy. 

In [7]:
import argparse
import numpy as np
import pickle
import gzip
import matplotlib.pyplot as plt


class Network:
    def __init__(self, sizes, keep_prob=-1):
        """
        sizes is a list : [input_dimensions, hidden_layer_dimensions, output_dimensions]
        """
        self.L = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(n, 1) for n in self.sizes[1:]]
        self.weights = [np.random.randn(n, m) for (
            m, n) in zip(self.sizes[:-1], self.sizes[1:])]
        self.keep_prob = keep_prob
        self.acc_train_array = []
        self.acc_test_array = []

    def g(self, z):
        """
        activation function
        """
        return sigmoid(z)

    def g_prime(self, z):
        """
        derivative of activation function
        """
        return sigmoid_prime(z)

    def forward_prop(self, a):
        """
        memory aware forward propagation for testing
        only.  back_prop implements it's own forward_prop
        """
        #YOUR CODE HERE
        a_list = [a]
        z_list = [np.zeros(a.shape)] 

        for W, b in zip(self.weights, self.biases):
            a = self.g(z)
            a_list.append(a)
        
    def grad_cost(self, a, y):
        """
        gradient of cost function
        Assumes C(a,y) = (a-y)^2/2
        """
        return (a - y)
    
    def log_train_progress(self, train, test, epoch):
        """ Logs training progres. 
        """
        acc_train = self.evaluate(train)
        self.acc_train_array.append(acc_train)
        if test is not None:
            acc_test = self.evaluate(test)
            self.acc_test_array.append(acc_test)
            print("Epoch {:4d}: Train {:10.5f}, Test {:10.5f}".format(
                epoch+1, acc_train, acc_test))
        else:
            print("Epoch {:4d}: Train {:10.5f}".format(
                epoch+1, acc_train))
            

    def SGD_train(self, train, epochs, eta, lam=0.0, verbose=True, test=None):
        """
        SGD for training parameters
        epochs is the number of epocs to run
        eta is the learning rate
        lam is the regularization parameter
        If verbose is set will print progressive accuracy updates
        If test set is provided, routine will print accuracy on test set as learning evolves
        """
        n_train = len(train)
        for epoch in range(epochs):
            perm = np.random.permutation(n_train)
            for kk in range(n_train):
                self.SGD_step(*train[perm[kk]], eta, lam)
            if verbose and epoch == 0 or (epoch + 1) % 20 == 0:
                self.log_train_progress(train, test, epoch)
    
    def SGD_step(self, x, y, eta, lam):
        """
        TODO: get gradients with x, y and do SGD on weights and biases

        Args:
            x: single sample features.
            y: single sample target.
            eta: learning rate.
            lam: Regularization parameter.
                
        """
        # TODO: get gradients with xk, yk and do SGD on weights and biases
        # YOUR CODE HERE
        # w'=w-ndL/dWk
        (dW_list, db_list) = back_prop(x, y)
        self.weights
        

    def back_prop(self, x, y):
        """
        Back propagation for derivatives of C wrt parameters
        """
        db_list = [np.zeros(b.shape) for b in self.biases]
        dW_list = [np.zeros(W.shape) for W in self.weights]
        
        a = x
        a_list = [a]
        z_list = [np.zeros(a.shape)]  # Pad with a placeholder so that indices match

        for W, b in zip(self.weights, self.biases):
            z = np.dot(W, a) + b
            z_list.append(z)
            a = self.g(z)
            a_list.append(a)

        # Back propagate deltas to compute derivatives
        # The following list gives hints on how to do it
        # calculating delta (Error) for the output layer
        # for the appropriate layers compute db_list[ell], dW_list[ell], delta
        delta_k = a_list[len(self.L -1)]
        for l in range(len(self.L)-1, 0, -1):
            dW_list[l] = np.dot(delta_k, np.transpose(a_list[l-1]))
            db_list[l] = delta_k
            delta_k = np.dot(np.transpose(self.weights[l]), g_prime(z_list[l-1]))
        
        return (dW_list, db_list)
    
    def back_prop_dropout(self, x, y):
        """
        Back propagation with dropout on the hidden layers other than the output layer.
        
        Dropout layer can be thought of as a special linear layer between layers.
        """
        db_list = [np.zeros(b.shape) for b in self.biases]
        dW_list = [np.zeros(W.shape) for W in self.weights]
        
        a = x
        a_list = [a]
        z_list = [np.zeros(a.shape)]  # Pad with a placeholder so that indices match
        # TODO: implement dropout using self.keep_prob
        # YOUR CODE HERE
        raise NotImplementedError()
        return (dW_list, db_list)

    def evaluate(self, test):
        """
        Evaluate current model on labeled test data
        """
        ctr = 0
        for x, y in test:
            yhat = self.forward_prop(x)
            ctr += yhat.argmax() == y.argmax()
        return float(ctr) / float(len(test))


def sigmoid(z, threshold=20):
    z = np.clip(z, -threshold, threshold)
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z):
    return sigmoid(z) * (1.0 - sigmoid(z))


def mnist_digit_show(flatimage, outname=None):

    import matplotlib.pyplot as plt

    image = np.reshape(flatimage, (-1, 14))

    plt.matshow(image, cmap=plt.cm.binary)
    plt.xticks([])
    plt.yticks([])
    if outname:
        plt.savefig(outname)
    else:
        plt.show()

### Part 1 [15 points]
Implement `SGD_step`, `back_prop`, and `forward_prop`. Use the following test cases to verify if the code is working as expected.

In [8]:
from tests import tests
tests.run_test_suite('prob 3', Network)

TestBackPropWithoutRegularization (tests.tests.TestNetwork) ... ERROR
TestBackPropWithRegularization (tests.tests.TestNetwork) ... ERROR

ERROR: TestBackPropWithoutRegularization (tests.tests.TestNetwork)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\reesl\Dropbox\School\csci4622\hw3\tests\tests.py", line 45, in TestBackPropWithoutRegularization
    nn_noreg.SGD_train(self.train, epochs=5, eta=0.25, lam=0.0, verbose=False)
  File "<ipython-input-7-cc4db12e3345>", line 82, in SGD_train
    self.SGD_step(*train[perm[kk]], eta, lam)
  File "<ipython-input-7-cc4db12e3345>", line 100, in SGD_step
    (dW_list, db_list) = back_prop(x, y)
NameError: name 'back_prop' is not defined

ERROR: TestBackPropWithRegularization (tests.tests.TestNetwork)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\reesl\Dropbox\School\csci4622\hw3\tests\tests.py", l

AssertionError: one or more tests for prob 3 failed

### Part 2 [8 points]

Run the above Network on MNIST Dataset and report the following (feel free to experiment with different learning rates).

* Change the hidden layer dimensions and experiment with these values: [5, 10, 20].
* Plot accuracies of different hidden layer dimensions vs. epochs for both training and testing.
* Explain the effect of hidden layer dimension on performance. 

**Note:** Accuracies are stored in `self.acc_train_array` and `self.acc_test_array` if `verbose` is `True`.

In [ ]:
location = './data/tinyMNIST.pkl.gz'
f = gzip.open(location, 'rb')
u = pickle._Unpickler(f)
u.encoding = 'latin1'
train, test = u.load()
input_dimensions = len(train[0][0])
output_dimensions = len(train[0][1])
print('Number of Input Features: ', input_dimensions)
print('Number of Output classes: ', output_dimensions)

nns = []
for hidden_layer_dimensions in [5, 10, 20]:
    print('\nHidden Layer Dimensions: ', hidden_layer_dimensions)
    nn = Network([input_dimensions, hidden_layer_dimensions, output_dimensions])
    nns.append(nn)
    nn.SGD_train(train, epochs=200, eta=0.1, lam=0.0001, verbose=True, test=test)

Plot training and testing accuracies below for each configuration. Use solid lines for testing accuracy, and dotted lines for train accuracy (`ls='dashed`). 

Be sure to label your lines and match colors accordingly.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
epochs_array = [i for i in range(1, 220, 20)]

# YOUR CODE HERE
raise NotImplementedError()


# YOUR CODE HERE
raise NotImplementedError()

ax.legend(loc="lower right", fontsize=16)
ax.set_xlabel("epochs", fontsize=16)
ax.set_ylabel("accuracy", fontsize=16)
plt.show()

Give your explanation below.

YOUR ANSWER HERE

**Extra Credit [10 points]:** Implement dropout by filling the `back_prop_dropout` function and update the `SGD_train` function to use it. Explain the impact of dropout on performance.

In [ ]:
from tests import tests
tests.run_test_suite('prob 4', Network)

In [ ]:
hidden_layer_dimensions = 60
nn = Network([input_dimensions, hidden_layer_dimensions, output_dimensions], keep_prob=0.5)
nn.SGD_train(train, epochs=400, eta=0.1, lam=0, verbose=True, test=test)

YOUR ANSWER HERE

## [9 points] Problem 4 - Tensors and Autograd.
In this problem, we will demonstrate the benefits of PyTorch's Autograd mechanics by converting your implementation from Problem to utilize PyTorch Tensors. This problem is not meant to be difficult, rather it should be quite simple once you've done Problem 3.

Note that there are *better* ways to go about this which we will cover next. Here. we are keeping things as close to numpy as possible on purpose to show what's going on behind the scenes. Take a moment to look through our modifications to get an understanding of the differences. In the comments, we'll make notes of native torch functions that perform the same operations. 

Here, SGD is implemented manually to show you what's going on behind the scenes, but  usually you'd use [`torch.optim.SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD). For future reference, the `weight_decay` parameter implements L2 normilazion. 

--- 

Before we move forward, let's cover a couple of points on notation. 

Just as it's best practice to do `import numpy as np`, PyTorch has best practices for imports as well:

```python
import torch 
import torch.nn as nn
import torch.nn.functional as F
```
You'll find that many of the functions you've been learning in NumPy have direct translations in PyTorch. The documentation can be found [here](https://pytorch.org/docs/stable/index.html) and is very user friendly, but we'll cover a few notable differences. 

- The numpy parameter `axis` becomes `dim` in PyTorch.
- Tensors have [view](https://pytorch.org/docs/stable/tensor_view.html) operations, which avoid copying data unnecasarily. Use these instead of `reshape()`. 
- Using `torch.from_numpy()` also avoids copying data stored in `numpy` arrays.
- Numpy interprets `np.dot()` of 2D matrices as matrix multiplication. Pytorch does not have this behaviour, see instead [`torch.matmul()`](https://pytorch.org/docs/stable/generated/torch.matmul.html?highlight=matmul#torch.matmul)

---
Now onto the problem, your tasks are as follows:

1. Modify `forward_prop` to work on PyTorch Tensors.
2. Modify `SGD_step` to work on PyTorch Tensors. This is easier than it sounds, since with PyTorch we don't have to worry about calculating gradients manually.

In [9]:
import torch


class TorchNetwork(Network):
    def __init__(self, sizes, keep_prob=-1):
        super().__init__(sizes, keep_prob)
        
        # Convert your weights and biases to Tensors.
        # The flag requires_grad=True tells PyTorch to track gradients for us. 
        self.biases = [torch.tensor(b, requires_grad=True) for b in self.biases]
        self.weights = [torch.tensor(w, requires_grad=True) for w in self.weights]
        

    def g(self, z):
        """ Activation function
            Could also use torch.sigmoid(z)
        """
        return sigmoid_torch(z) 

    def forward_prop(self, a):
        """ Memory aware forward propagation. 
            Now this is for both training and testing.
        
        """
        # TODO: Convert your forward_prop function from part a to work on Tensors. 
        # YOUR CODE HERE
        raise NotImplementedError()

    def cost(self, a, y):
        """ Cost function, same as F.mse_loss()
        """
        return ((a - y)**2).mean() 
            

    def SGD_step(self, x, y, eta, lam):
        """
            Performs a single step of SGD on weights and biases

        Args:
            x: single sample features.
            y: single sample target.
            eta: learning rate.
            lam: Regularization parameter.
                
        """
        
        # Zero out the old gradients if they exist. 
        # Would be optim.zero_grad() if we were using an optimizer.
        for p in (*self.weights, *self.biases):
            if p.grad is not None:
                p.grad.detach_()
                p.grad.zero_()
            
        # TODO: 
        # (1) call forward_prop to get a. 
        # (2) Calucale loss using the cost function.
        # (3) Use loss.backward() to to perform backpropagation automatically.

        # YOUR CODE HERE
        raise NotImplementedError()

        # Perform sgd update 
        # Would be optimizer.step()
        with torch.no_grad():
            # l2 regularization on weights only.
            for p in self.weights:
                p.grad.add_(p, alpha=lam)
            # sgd update. 
            for p in (*self.weights, *self.biases):
                p.add_(p.grad, alpha=-eta)
                

def sigmoid_torch(z, threshold=20):
    """ Same as sigmoid before, now with torch.clamp() instead of np.clip().
        We could also just use torch.sigmoid().
    """
    z = torch.clamp(z, -threshold, threshold)
    return 1.0 / (1.0 + torch.exp(-z))


**Part 5 [10 points]:** Implement `SGD_step` and `forward_prop`. Use the following test cases to verify if the code is working as expected.

In [10]:
from tests import tests
tests.run_test_suite('prob 5', TorchNetwork)

TestBackPropWithoutRegularization (tests.tests.TestNetwork) ... ERROR
TestBackPropWithRegularization (tests.tests.TestNetwork) ... ERROR

ERROR: TestBackPropWithoutRegularization (tests.tests.TestNetwork)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\reesl\Dropbox\School\csci4622\hw3\tests\tests.py", line 45, in TestBackPropWithoutRegularization
    nn_noreg.SGD_train(self.train, epochs=5, eta=0.25, lam=0.0, verbose=False)
  File "<ipython-input-7-cc4db12e3345>", line 82, in SGD_train
    self.SGD_step(*train[perm[kk]], eta, lam)
  File "<ipython-input-9-183b3d232823>", line 60, in SGD_step
    raise NotImplementedError()
NotImplementedError

ERROR: TestBackPropWithRegularization (tests.tests.TestNetwork)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\reesl\Dropbox\School\csci4622\hw3\tests\tests.py", line 75, in TestBackPropWithRegula

AssertionError: one or more tests for prob 5 failed

[23 Points] Problem 5 - Implement RNN Network to classify whether text is spam or ham 
---

Dataset is obtained from UCI Machine Learning repository consisting of SMS tagged messages (labelled as either **ham** (legitimate) or **spam**) that have been collected for SMS Spam research.

We will now use [PyTorch](https://pytorch.org/docs/stable/index.html) to implement a classifier. Update the snippet below to build a model with an embedding layer, and an LSTM layer, and a dense layer. This question allows you to get familiar with popular deep learning toolkits and the solution only has a few lines. In practice, there is no need to reinvent the wheels.


Learn more about RNN : https://colah.github.io/posts/2015-08-Understanding-LSTMs/

**Note**: You need to install Tensorflow for this problem (CPU version if you do not have a nvidia GPU). You can find installation instructions [here](https://www.tensorflow.org/install)

If you are using Anaconda, install the CPU version by doing 
```
conda install 'tensorflow=*=mkl*'
```

In [2]:
import multiprocessing as mp
import pickle
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from torch.utils.data import DataLoader, TensorDataset

import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger

class SpamHam(pl.LightningDataModule):
    """ A datamodule for the RNN. You shouldn't have to modify this class."""
    def __init__(self, dict_size=5000, example_length=150, batch_size=128):
        super().__init__()

        self.dict_size = dict_size
        self.batch_size = batch_size
        self.example_length = example_length

        # preprocess training data
        self.tok = Tokenizer(num_words=dict_size)

    def load_data(self, location):
        return pickle.load(open(location, 'rb'))

    def make_dataset(self, x, y, tok=None):
        # tokenize
        sequences = self.tok.texts_to_sequences(x)
        x = sequence.pad_sequences(sequences, maxlen=self.example_length)

        # make torch arrays.
        x = torch.from_numpy(x).to(torch.int64)
        y = torch.from_numpy(y).to(torch.float32)

        return TensorDataset(x, y)

    def setup(self, stage):
        # load data
        train_x, test_x, train_y, test_y = self.load_data('./data/spam_data.pkl')
        # fit tokenizer
        self.tok.fit_on_texts(train_x)
        # make datasets
        self.train = self.make_dataset(train_x, train_y)
        self.test = self.make_dataset(train_x, train_y)

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True,
                          num_workers=mp.cpu_count() // 4)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, shuffle=False,
                          num_workers=mp.cpu_count() // 4)

### Part 1 [15 points]

Complete the functions `init`, `training_step`, and `validation_step` and `forward`functions. 

In [ ]:
class RNN(pl.LightningModule):
    """ RNN classifier """

    def __init__(self, embedding_length=32, dict_size=5000, example_length=150):
        """
        initialize RNN model
        :param embedding_length: size of word embedding
        """
        super().__init__()

        # TODO: build model with Embedding, LSTM and dense layers.
        # Documentation for LSTM layer in :
        #     https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM
        # Note there are multiple ways to implement your model, 
        # we suggest adding individual layers here, but any method is fine.

            
        # YOUR CODE HERE
        raise NotImplementedError()

        self.example_input_array = torch.zeros([1, 150], dtype=torch.int64)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def forward(self, x):
        """ Implement a forward call your model here."""
        # YOUR CODE HERE
        raise NotImplementedError()
        return out

    def accuracy(self, y_hat, y):
        return (y == y_hat.round()).to(torch.float32).mean()

    def training_step(self, batch, batch_idx):
        """ Perfom a training step.
        
        TODO:
            - forward pass on data in batch
            - compute training loss.
            - Compute training accuracy 
        """
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # We implemented logging for you. 
        result = pl.TrainResult(loss)
        result.log('train_loss', loss)
        result.log('train_accuracy', acc, prog_bar=True)
        return result

    def test_step(self, batch, batch_idx):
        """ Perfom a test step 
            hint: your code should be the same as your train step
        
        TODO:
            - forward pass on data in batch
            - compute test loss.
            - Compute test accuracy  
        """
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
        # We implemented logging for you. 
        result = pl.EvalResult(loss)
        result.log('test_loss', loss, prog_bar=True)
        result.log('test_acc', acc, prog_bar=True)

        return result


In [ ]:
# Define your data 
spam_ham_dm = SpamHam()

def run_rnn(**kwargs):
    # helper function for running RNN.
    logger = CSVLogger("logs", name="rnn")
    trainer = pl.Trainer(
        gpus=int(torch.cuda.is_available()),
        logger=logger,
        min_epochs=5,
        max_epochs=5,
        row_log_interval=1,
        log_save_interval=1
    )
    
    model = RNN(**kwargs)
    trainer.fit(model, spam_ham_dm)
    results = trainer.test()
    return results, logger.experiment.metrics_file_path

results, _ = run_rnn()
print('Accuracy for LSTM: ', results[0]['test_acc'])

YOUR ANSWER HERE

### Part 2 [8 points]
* Change the embedding length and experiment with these values: [8, 16, 32, 48, 64].
* Plot training accuracies of different embedding lengths vs. epochs.
* Observe and explain the impact of embedding length in LSTM Model.

In [ ]:
mfiles = []
test_accuracy_array = []

for embedding_len in [8, 16, 32, 48, 64]:    
    results, metrics = run_rnn(embedding_length=embedding_len)
    print('Accuracy for LSTM: ', results[0]['test_acc'])
    mfiles.append(metrics)

train_accuracy_matrix = np.array(
    [pd.read_csv(f)["train_accuracy"].values for f in mfiles])

Plot training results below.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Give your observation and explanation below.

YOUR ANSWER HERE